In [ ]:
!pip install -q kaggle

In [ ]:
!nvidia-smi

Wed Jun  2 05:00:04 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import files
files.upload()

Saving kaggle (1).json to kaggle (1).json


{'kaggle (1).json': b'{"username":"bommakantiharshitha","key":"12570043671b5aeb3634958a96f03717"}'}

In [ ]:
!mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c turnkey-summer-interns-idl-hw1p2

 96% 298M/310M [00:05<00:00, 48.6MB/s]
100% 310M/310M [00:05<00:00, 58.4MB/s]
  0% 0.00/518 [00:00<?, ?B/s]
100% 518/518 [00:00<00:00, 465kB/s]
100% 3.52G/3.52G [01:06<00:00, 24.5MB/s]
100% 3.52G/3.52G [01:06<00:00, 56.6MB/s]
 99% 320M/322M [00:07<00:00, 33.3MB/s]
100% 322M/322M [00:07<00:00, 45.5MB/s]
  0% 0.00/3.43M [00:00<?, ?B/s]
100% 3.43M/3.43M [00:00<00:00, 233MB/s]
  0% 0.00/500k [00:00<?, ?B/s]
100% 500k/500k [00:00<00:00, 69.0MB/s]
  0% 0.00/508k [00:00<?, ?B/s]
100% 508k/508k [00:00<00:00, 72.6MB/s]
 97% 5.00M/5.16M [00:00<00:00, 28.8MB/s]
100% 5.16M/5.16M [00:00<00:00, 25.3MB/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
path='/content/drive/MyDrive/Speech_recognization/'
train_data=np.load(path+'train.npy',allow_pickle=True)
train_label=np.load(path+'train_labels.npy',allow_pickle=True)
d_data=np.load(path+'dev.npy',allow_pickle=True)
d_labels=np.load(path+'dev_labels.npy',allow_pickle=True)

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import time
import os


In [ ]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, X, Y, context):
        # Store paramters as class variables
        self.X=X
        self.Y=Y
        self.context=context
        # Taking the data in variable x and arraning all the frames in one list
        t_data=[]
        for x in X:
          for i in x:
            t_data.append(i)
        
        self.X=np.array(t_data,dtype=np.float)
     
        self.X=np.pad(self.X,pad_width=((self.context,self.context),(0,0)),mode="constant",constant_values=0)
        t_data=[]
        for y in Y:
          for j in y:
            t_data.append(j)
        self.Y=np.array(t_data,dtype=np.float)
        # print("self.Y at index 0 istype is:",self.Y[0])
        
         
            
        
    def __len__(self):
        # print(self.Y.shape[0])
        return len(self.Y)

    def __getitem__(self,index):

        
        start_index=index
        end_index=index+2*self.context+1
        xx=self.X[start_index:end_index,:].flatten()
        yy=self.Y[index]
        return xx,yy
    def collate_fn(batch):
        
        ### Select all data from batch (1 line)
        batch_x = [x for x,y in batch]
        # print(batch_x)
        ### Select all labels from batch (1 line)
        batch_y = [y for x,y in batch]
        
        ### Convert batched data and labels to tensors (2 lines)
        batch_x = torch.as_tensor(batch_x)
        batch_y = torch.as_tensor(batch_y)
        
        ### Return batched data and labels (1 line)
        return batch_x, batch_y


In [ ]:
cuda = torch.cuda.is_available()

num_workers=2 if cuda else 0
context=5
dataset=MyDataset(train_data,train_label,context)
train_loader_args = dict(shuffle=True, batch_size=256, num_workers=num_workers, pin_memory=True) if cuda\
                    else dict(shuffle=True, batch_size=64)
train_loader=torch.utils.data.DataLoader(dataset,**train_loader_args)

In [ ]:
import torch.nn as nn
class Simple_MLP(nn.Module):
    def __init__(self,input_output_data):
        super(Simple_MLP, self).__init__()
        self.input_output_data=input_output_data
        layers=[]
        for i in range(len(self.input_output_data)-2):
          layers.append(nn.Linear(input_output_data[i],input_output_data[i+1]))
          layers.append(nn.BatchNorm1d(input_output_data[i+1]))
          layers.append(nn.ReLU())
          layers.append(nn.Dropout(p=0.168))
        layers.append(nn.Linear(input_output_data[-2],input_output_data[-1]))
        
        self.net=nn.Sequential(*layers)

    def forward(self, x):
      x = torch.flatten(x,start_dim = 1)
      return self.net(x.float())
        # make sure to return the output after 
        # call the network created above

In [ ]:
#print(context)
cuda = torch.cuda.is_available()


In [ ]:
import torch.optim as optim
context=5
k=(context*2+1)*40
model=Simple_MLP([k,3072,2048,1024,1024,512,128,71])
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters())  # model.parameters refer to the network’s weights
device=torch.device("cuda" if cuda else "cpu")
model.to(device)

Simple_MLP(
  (net): Sequential(
    (0): Linear(in_features=440, out_features=3072, bias=True)
    (1): BatchNorm1d(3072, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.168, inplace=False)
    (4): Linear(in_features=3072, out_features=2048, bias=True)
    (5): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.168, inplace=False)
    (8): Linear(in_features=2048, out_features=1024, bias=True)
    (9): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Dropout(p=0.168, inplace=False)
    (12): Linear(in_features=1024, out_features=1024, bias=True)
    (13): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
    (15): Dropout(p=0.168, inplace=False)
    (16): Linear(in_features=1024, out_features=512, bias=True)
    (17): BatchNorm1d(512, eps=1e-05, momentum=0.1

In [ ]:

def train_epoch(model, train_loader, criterion, optimizer):

  print("Training....") 
  model.train()

  running_loss=0.0
  total_predictions=0.0 
  correct_predictions=0.0
  start_time=time.time()

  for batch_idx, (data, target) in enumerate(train_loader):
    print(batch_idx)
    optimizer.zero_grad() 
    data=data.to(device)
  
    target=target.to(device) 

    outputs=model(data) 
    loss=criterion(outputs, target.long())
    running_loss+=loss.item()

    loss.backward()    # we calculate the gradients by calling the backward() function
    optimizer.step()  # is used to update the weights using the calculated gradients.
    
    
    # Here the accuracy can be calculaated with the no training examples being 
    # correclty predicted i,e target is the y and outputs is y^)
    

    value,index=torch.max(outputs,1)
    correct_predictions=torch.sum(index==target)
    total_predictions=outputs.shape[0]
    
    end_time=time.time()
    
    running_loss/=len(train_loader)
    
    print("Training Loss:", running_loss, "Time: ",end_time-start_time,'s')
    
    acc=(correct_predictions/total_predictions)*100.0
 

    
    print('Training Accuracy:', acc, "% ") 
    print("=====================================================================")
  return running_loss ,acc

In [ ]:

train_epoch(model, train_loader, criterion, optimizer)


Streaming output truncated to the last 5000 lines.
Training Accuracy: tensor(72.2656, device='cuda:0') % 
69469
Training Loss: 1.2688536270139553e-05 Time:  690.2836263179779 s
Training Accuracy: tensor(72.2656, device='cuda:0') % 
69470
Training Loss: 1.425701072532527e-05 Time:  690.2926480770111 s
Training Accuracy: tensor(66.7969, device='cuda:0') % 
69471
Training Loss: 1.3696229518077753e-05 Time:  690.3021876811981 s
Training Accuracy: tensor(68.3594, device='cuda:0') % 
69472
Training Loss: 1.4279892392271617e-05 Time:  690.3126876354218 s
Training Accuracy: tensor(69.5312, device='cuda:0') % 
69473
Training Loss: 1.1797244014534528e-05 Time:  690.323798418045 s
Training Accuracy: tensor(75., device='cuda:0') % 
69474
Training Loss: 1.431802296977875e-05 Time:  690.3337349891663 s
Training Accuracy: tensor(70.7031, device='cuda:0') % 
69475
Training Loss: 1.4143302348647468e-05 Time:  690.3434309959412 s
Training Accuracy: tensor(67.1875, device='cuda:0') % 
69476
Training Loss

KeyboardInterrupt: ignored

In [ ]:
model_save_name = 'trainmodel1.pt'
path = '/content/drive/MyDrive/Speech_recognization/trainmodel1.pt' 
torch.save(model.state_dict(), path)

In [ ]:

model.load_state_dict(torch.load('/content/drive/MyDrive/Speech_recognization/trainmodel1.pt'))



<All keys matched successfully>

In [ ]:
cuda = torch.cuda.is_available()

num_workers=2 if cuda else 0
context= 5
val_dataset=MyDataset(d_data,d_labels,context)
val_loader_args = dict(shuffle=False, batch_size=256, num_workers=num_workers, pin_memory=True) if cuda\
                    else dict(shuffle=False, batch_size=64)
val_loader=torch.utils.data.DataLoader(val_dataset,**val_loader_args)



In [ ]:
def val_epoch(model, val_loader, criterion):
  print("validating....") 
  model.eval()

  running_loss=0.0
  total_predictions=0.0 
  correct_predictions=0.0
  start_time=time.time()
  for batch_idx, (data, target) in enumerate(val_loader):
    print(batch_idx)
 
    data=data.to(device)

    target=target.to(device) #data & sodel on same device

    outputs=model(data) 
    loss=criterion(outputs, target.long())
    running_loss+=loss.item()

    
    
    
    # Here the accuracy can be calculaated with the no training examples being 
    # correclty predicted i,e target is the y and outputs is y^)


    value,index=torch.max(outputs,1)
    correct_predictions=torch.sum(index==target)
    total_predictions=outputs.shape[0]
    
    end_time=time.time()

    running_loss/=len(val_loader)

    print("testing Loss:", running_loss, "Time: ",end_time-start_time,'s')
    acc=(correct_predictions/total_predictions)*100.0
   

    print('Testing Accuracy:', acc, "% ") 
  return running_loss ,acc
    
    

In [ ]:
val_epoch(model, val_loader, criterion)

Streaming output truncated to the last 5000 lines.
Testing Accuracy: tensor(71.0938, device='cuda:0') % 
1244
testing Loss: 0.00011885396095951989 Time:  8.63820505142212 s
Testing Accuracy: tensor(75.7812, device='cuda:0') % 
1245
testing Loss: 0.00012289562828312085 Time:  8.644039869308472 s
Testing Accuracy: tensor(75.3906, device='cuda:0') % 
1246
testing Loss: 0.0001475939769313242 Time:  8.651000261306763 s
Testing Accuracy: tensor(69.1406, device='cuda:0') % 
1247
testing Loss: 7.298398632455311e-05 Time:  8.657231569290161 s
Testing Accuracy: tensor(86.7188, device='cuda:0') % 
1248
testing Loss: 0.00019435181588089214 Time:  8.66486668586731 s
Testing Accuracy: tensor(60.9375, device='cuda:0') % 
1249
testing Loss: 0.00015954204000206054 Time:  8.672013282775879 s
Testing Accuracy: tensor(67.1875, device='cuda:0') % 
1250
testing Loss: 0.00024204054396316304 Time:  8.678234815597534 s
Testing Accuracy: tensor(56.2500, device='cuda:0') % 
1251
testing Loss: 0.00016202741066243

Exception in thread Thread-13:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/_utils/pin_memory.py", line 25, in _pin_memory_loop
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 113, in get
    return _ForkingPickler.loads(res)
  File "/usr/local/lib/python3.7/dist-packages/torch/multiprocessing/reductions.py", line 282, in rebuild_storage_fd
    fd = df.detach()
  File "/usr/lib/python3.7/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/usr/lib/python3.7/multiprocessing/resource_sharer.py", line 87, in get_connection
    c = Client(address, authkey=process.current_process().authkey)
  File "/usr/lib/pytho

KeyboardInterrupt: ignored

In [ ]:
import torch

In [ ]:
test_data=np.load("/content/drive/MyDrive/Speech_recognization/test.npy" , allow_pickle=True)

In [ ]:
class MyDatasettest(torch.utils.data.Dataset):
    def __init__(self, X, context):
        # Store paramters as class variables
        self.X=X
       
        self.context=context
        # Taking the data in variable x and arraning all the frames in one list
        tot_data=[]
        for x in X:
          for i in x:
            tot_data.append(i)
        
        self.X=np.array(tot_data,dtype=np.float)
      #  padding the data at starting of the data and ending of  the data with context no of rows 
        self.length=self.X.shape[0]
        self.X=np.pad(self.X,pad_width=((self.context,self.context),(0,0)),mode="constant",constant_values=0)
    
            
        
    def __len__(self):
        # print(self.Y.shape[0])
        return self.length

    def __getitem__(self,index):

        
        start_index=index
        end_index=index+2*self.context+1
        xx=self.X[start_index:end_index,:].flatten()
        # print(xx)
        # xx=self.X[index]
        # yy=self.Y[index]
        return xx
    def collate_fn(batch):
        
        ### Select all data from batch (1 line)
        batch_x =batch
        # print(batch_x)
        ### Select all labels from batch (1 line)
        # batch_y = [y for x in batch]
        
        ### Convert batched data and labels to tensors (2 lines)
        batch_x = torch.as_tensor(batch_x)
        # batch_y = torch.as_tensor(batch_y)
        
        ### Return batched data and labels (1 line)
        return batch_x
        

In [ ]:
cuda = torch.cuda.is_available()

num_workers=2 if cuda else 0
context=5
dataset=MyDatasettest(test_data,context)
train_loader_args = dict(shuffle=False, batch_size=256, num_workers=num_workers, pin_memory=True) if cuda\
                    else dict(shuffle=False, batch_size=64)
test_loader=torch.utils.data.DataLoader(dataset,**train_loader_args)


In [ ]:
import numpy as np

In [ ]:
model

Simple_MLP(
  (net): Sequential(
    (0): Linear(in_features=1240, out_features=3072, bias=True)
    (1): BatchNorm1d(3072, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.168, inplace=False)
    (4): Linear(in_features=3072, out_features=2048, bias=True)
    (5): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.168, inplace=False)
    (8): Linear(in_features=2048, out_features=1024, bias=True)
    (9): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Dropout(p=0.168, inplace=False)
    (12): Linear(in_features=1024, out_features=1024, bias=True)
    (13): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
    (15): Dropout(p=0.168, inplace=False)
    (16): Linear(in_features=1024, out_features=512, bias=True)
    (17): BatchNorm1d(512, eps=1e-05, momentum=0.

In [ ]:
model=Simple_MLP([k,2048,1024,512,512,256,128,71])
model=Simple_MLP([k,3072,2048,1024,1024,512,128,71])

In [ ]:
print(state_dict.keys())

odict_keys(['net.0.weight', 'net.0.bias', 'net.1.weight', 'net.1.bias', 'net.1.running_mean', 'net.1.running_var', 'net.1.num_batches_tracked', 'net.4.weight', 'net.4.bias', 'net.5.weight', 'net.5.bias', 'net.5.running_mean', 'net.5.running_var', 'net.5.num_batches_tracked', 'net.8.weight', 'net.8.bias', 'net.9.weight', 'net.9.bias', 'net.9.running_mean', 'net.9.running_var', 'net.9.num_batches_tracked', 'net.12.weight', 'net.12.bias', 'net.13.weight', 'net.13.bias', 'net.13.running_mean', 'net.13.running_var', 'net.13.num_batches_tracked', 'net.16.weight', 'net.16.bias', 'net.17.weight', 'net.17.bias', 'net.17.running_mean', 'net.17.running_var', 'net.17.num_batches_tracked', 'net.20.weight', 'net.20.bias', 'net.21.weight', 'net.21.bias', 'net.21.running_mean', 'net.21.running_var', 'net.21.num_batches_tracked', 'net.24.weight', 'net.24.bias'])


In [ ]:
def test_epoch(model, test_loader):

  print("testing data is taken....") 
  model.eval()
  
  
  
  result=[]
  for batch_idx, data in enumerate(test_loader):
    data=data.to(device)
    outputs=model(data) 
    # print(outputs.shape)
    value,index=torch.max(outputs,1)
    result.append(index.tolist())
  return result  
    
  

    
 
result=test_epoch(model, test_loader)

testing data is taken....


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
print(result[0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 68, 68, 68, 68, 68, 68, 64, 64, 64, 64, 64, 64, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 40, 40, 40, 40, 40, 40, 40, 45, 45, 45, 45, 64, 64, 64, 0, 0, 22, 22, 6, 6, 6, 6, 6, 6, 6, 31, 31, 31, 31, 31, 18, 18, 31, 31, 46, 43, 43, 21, 21, 21, 21, 57, 57, 57, 57, 57, 57, 57, 57, 57, 8, 36, 36, 36, 36, 37, 37, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 45, 45, 45, 45, 45, 18, 18, 6, 6, 25, 18, 18, 6, 6, 6, 6, 6, 15, 15, 15, 15, 15, 15, 15, 15, 15, 46, 46, 46, 46, 43, 43, 43, 43, 43, 43, 43, 8, 8, 8, 8, 8, 49, 49, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 49, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 25, 25, 25, 25, 25, 25, 25, 25]


In [ ]:
import torch

In [ ]:
tensor_result=torch.as_tensor(result[0:-1]).flatten()
tensor_result.shape
last=torch.as_tensor(result[-1])
final_tensor=torch.hstack((tensor_result,last))
final=final_tensor.tolist()
from pandas import DataFrame
# your_list = ['item1', 'item2', 'item3',...]
df = DataFrame (final,columns=['label'])
df.to_csv("Result.csv",index_label='id')
df.head

<bound method NDFrame.head of          label
0            0
1            0
2            0
3            0
4            0
...        ...
1618830      0
1618831      0
1618832      0
1618833      0
1618834      0

[1618835 rows x 1 columns]>

In [ ]:
!kaggle competitions submit -c turnkey-summer-interns-idl-hw1p2 -f Result.csv -m "Message"

100% 15.5M/15.5M [00:04<00:00, 3.40MB/s]
Successfully submitted to Turnkey Summer Interns IDL : HW1P2

In [ ]:
!kaggle competitions submit -c turnkey-summer-interns-idl-hw1p2 -f Drop_256ba_con15.csv -m "Message"

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
